link to hackathon  -  https://huggingface.co/traversaal-ai-hackathon

Things to Try:-

1. For Stage 1 we have to make a semantic RAG system that tells **why a hotel is good** so **summarisation** as well **prompt_temple** will be used now
2. Different embeddings models - **mixbread** and **bge** vecause they are state if the art as of now. - Mix bread large - https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1 and it's in top 5 of MTEB leaderboard - https://huggingface.co/spaces/mteb/leaderboard
3. Rerankers - **cohere**,**flashrank** and etc
4. Different Retrival stratergies
5. To improve RAG's performance we can go graph way(last option)  - https://blog.langchain.dev/enhancing-rag-based-applications-accuracy-by-constructing-and-leveraging-knowledge-graphs/
6. We can add feature of reverese image search via qdrant - Person uploads a photo if that photo is similar to ares api's output - we will look int our DB and answer - https://colab.research.google.com/github/qdrant/examples/blob/master/ecommerce_reverse_image_search/ecommerce-reverse-image-search.ipynb
7. For our own Judgement we can either use **MRR** or **precision@k** - https://qdrant.tech/documentation/tutorials/retrieval-quality/
8. Recommendation engine Qdrant - https://qdrant.tech/articles/new-recommendation-api/

In [1]:
# %pip install --upgrade --quiet langchain langchain-openai

In [8]:
import pandas as pd
from datasets import load_dataset
dataset = load_dataset("traversaal-ai-hackathon/hotel_datasets")
df=pd.DataFrame(dataset['train'])
df.to_csv('hotels-dataset.csv', index=False)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['hotel_name', 'hotel_description', 'review_title', 'review_text', 'rate', 'tripdate', 'hotel_url', 'hotel_image', 'price_range', 'rating_value', 'review_count', 'street_address', 'locality', 'country'],
        num_rows: 5997
    })
})

In [10]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings import HuggingFaceEmbeddings
import qdrant_client
from dotenv import load_dotenv
import os
load_dotenv()

True

In [11]:
dataset['train'][0]

{'hotel_name': 'Romance Istanbul Hotel',
 'hotel_description': 'Romance Istanbul Hotel has 39 rooms.Every room is elegantly furnished and harmonizes the modern life style with the traditional Ottoman touch. Romance Istanbul sits at the intersection of the old city’s most important part. With its luxuriously inspiring design and landmark old city location, steeped in the history of its surroundings, Romance Istanbul Hotel welcomes you with exceptional designed rooms and world-renowned Turkish hospitality. Our colleagues deliver the most personal service. It is perfectly placed and perfectly designed to enhance all that Istanbul has to offer. Each room offers a private bathroom and shower. Each is equipped with a satellite TV and free wifi connection. The rooms size change between 20 m2 and 45 m2. It includes 7 suite rooms: 1 Royal Suite, 4 Grand Suite, 1 Romance Suite and 1 Premium Suite, 2 Luxury Room With Terrace, 22 Deluxe Room, 8 City Room.',
 'review_title': 'An exceptional boutiqu

In [12]:
QDRANT_URL = os.getenv('QDRANT_URL')
QDRANT_API_KEY = os.getenv('QDRANT_API_KEY')
OPENAI_API_KEY = os.getenv('open_ai_key')
os.environ['openai_api_key'] = OPENAI_API_KEY

In [13]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_API_KEY,
)

In [14]:

VECTOR_DB_COLLECTION = 'traversal'

In [15]:
loader = CSVLoader(file_path='hotels-dataset.csv', encoding='utf')
data = loader.load()

In [16]:
type(data[0])

langchain_core.documents.base.Document

In [23]:
embeddings = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

In [24]:
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

In [25]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='mixedbread-ai/mxbai-embed-large-v1', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [26]:
# model

In [27]:
# index creation 

url = QDRANT_URL
api_key = QDRANT_API_KEY
qdrant = Qdrant.from_documents(
    data, #document type suppported by langchain
    embeddings, #hf embeddings
    url=url, #url for qdrant index
    prefer_grpc=True,
    api_key=api_key,
    collection_name=VECTOR_DB_COLLECTION,
    force_recreate=True
)

KeyboardInterrupt: 

In [14]:
query = "describe Romance Istanbul Hotel"
found_docs = qdrant.similarity_search(query)

In [15]:
found_docs[0]

Document(page_content='hotel_name: Romance Istanbul Hotel\nhotel_description: Romance Istanbul Hotel has 39 rooms.Every room is elegantly furnished and harmonizes the modern life style with the traditional Ottoman touch. Romance Istanbul sits at the intersection of the old city’s most important part. With its luxuriously inspiring design and landmark old city location, steeped in the history of its surroundings, Romance Istanbul Hotel welcomes you with exceptional designed rooms and world-renowned Turkish hospitality. Our colleagues deliver the most personal service. It is perfectly placed and perfectly designed to enhance all that Istanbul has to offer. Each room offers a private bathroom and shower. Each is equipped with a satellite TV and free wifi connection. The rooms size change between 20 m2 and 45 m2. It includes 7 suite rooms: 1 Royal Suite, 4 Grand Suite, 1 Romance Suite and 1 Premium Suite, 2 Luxury Room With Terrace, 22 Deluxe Room, 8 City Room.\nreview_title: Very nice vis

In [16]:
print(found_docs[0].page_content)

hotel_name: Romance Istanbul Hotel
hotel_description: Romance Istanbul Hotel has 39 rooms.Every room is elegantly furnished and harmonizes the modern life style with the traditional Ottoman touch. Romance Istanbul sits at the intersection of the old city’s most important part. With its luxuriously inspiring design and landmark old city location, steeped in the history of its surroundings, Romance Istanbul Hotel welcomes you with exceptional designed rooms and world-renowned Turkish hospitality. Our colleagues deliver the most personal service. It is perfectly placed and perfectly designed to enhance all that Istanbul has to offer. Each room offers a private bathroom and shower. Each is equipped with a satellite TV and free wifi connection. The rooms size change between 20 m2 and 45 m2. It includes 7 suite rooms: 1 Royal Suite, 4 Grand Suite, 1 Romance Suite and 1 Premium Suite, 2 Luxury Room With Terrace, 22 Deluxe Room, 8 City Room.
review_title: Very nice visit to Turkey
review_text:

In [17]:
query = "best reviews for Conrad Istanbul Bosphorus"
found_docs = qdrant.similarity_search(query)
print(found_docs[0].page_content)

hotel_name: Conrad Istanbul Bosphorus
hotel_description: Boasting a perfect location between 2 continents, indulge in a stay at Conrad Istanbul Bosphorus and discover yourself at one of the world's most alluring destinations. At the very centre of the city, yet far away from the hustle and bustle, Conrad Istanbul Bosphorus's rooms and suites allow for the highest standard of relaxation, complete with personalised comfort and services. Savor Mediterranean cuisine at Manzara, take in the view of the Bosphorus while lounging at the Summit Bar. Relax with Spa treatments, enjoy a revitalizing workout in the brand-new health club and clay tennis courts.
review_title: Best trip
review_text: Amazing!!!!! Good view and good stay! Very Good location!! Good food, excellent BED!!  In the room have a lot of space and it is very nice. I really love to come back and recommend to my friends and followers of course. Thank you for the hospitality Selim :)
rate: 5.0
tripdate: January 2024
hotel_url: http

## Starting querying

In [18]:
dataset['train'][0]

{'hotel_name': 'Romance Istanbul Hotel',
 'hotel_description': 'Romance Istanbul Hotel has 39 rooms.Every room is elegantly furnished and harmonizes the modern life style with the traditional Ottoman touch. Romance Istanbul sits at the intersection of the old city’s most important part. With its luxuriously inspiring design and landmark old city location, steeped in the history of its surroundings, Romance Istanbul Hotel welcomes you with exceptional designed rooms and world-renowned Turkish hospitality. Our colleagues deliver the most personal service. It is perfectly placed and perfectly designed to enhance all that Istanbul has to offer. Each room offers a private bathroom and shower. Each is equipped with a satellite TV and free wifi connection. The rooms size change between 20 m2 and 45 m2. It includes 7 suite rooms: 1 Royal Suite, 4 Grand Suite, 1 Romance Suite and 1 Premium Suite, 2 Luxury Room With Terrace, 22 Deluxe Room, 8 City Room.',
 'review_title': 'An exceptional boutiqu

In [19]:
query = 'hey can you tell me something about Romance Istanbul Hotel'
found_docs = qdrant.similarity_search(query)
found_docs

[Document(page_content='hotel_name: Romance Istanbul Hotel\nhotel_description: Romance Istanbul Hotel has 39 rooms.Every room is elegantly furnished and harmonizes the modern life style with the traditional Ottoman touch. Romance Istanbul sits at the intersection of the old city’s most important part. With its luxuriously inspiring design and landmark old city location, steeped in the history of its surroundings, Romance Istanbul Hotel welcomes you with exceptional designed rooms and world-renowned Turkish hospitality. Our colleagues deliver the most personal service. It is perfectly placed and perfectly designed to enhance all that Istanbul has to offer. Each room offers a private bathroom and shower. Each is equipped with a satellite TV and free wifi connection. The rooms size change between 20 m2 and 45 m2. It includes 7 suite rooms: 1 Royal Suite, 4 Grand Suite, 1 Romance Suite and 1 Premium Suite, 2 Luxury Room With Terrace, 22 Deluxe Room, 8 City Room.\nreview_title: Very nice vi

In [50]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage,AIMessage
from langchain.memory import ChatMessageHistory
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-1106",api_key = OPENAI_API_KEY)

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context} remember that you are awesome hotel advisor",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

In [51]:
query = 'can you pls share something about romance istanbul hotel and what are it\'s reviews and ratings'

#fetch only docs that are above 75%similarity search result
docs = qdrant.similarity_search(query,k=5,score_threshold=0.75)

In [52]:
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message(query)

result = document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)

In [53]:
result

'Romance Istanbul Hotel is a luxurious hotel with 39 elegantly furnished rooms that harmonize modern lifestyle with a traditional Ottoman touch. It is situated at the intersection of the old city’s most important part, making it an ideal location for exploring Istanbul\'s landmarks.\n\nThe hotel offers exceptional designed rooms with a private bathroom and shower, satellite TV, and free wifi connection. The room sizes vary between 20 m2 and 45 m2, and there are different types of suites and rooms available, including Royal Suite, Grand Suite, Romance Suite, Premium Suite, Luxury Room With Terrace, Deluxe Room, and City Room.\n\nWith a perfect blend of design and Turkish hospitality, Romance Istanbul Hotel has received glowing reviews from guests. The average rating is 5.0, and there are over 4023 reviews praising the hotel\'s exceptional service and ambiance.\n\nHere are some review highlights:\n- "Great stay in Osman atmosphere" - Guests loved the clean rooms, friendly staff, and deli

#### So we have a basic chat interface ready that works as a RAG

## Reasin why we need to improve

In [24]:
query = 'hey anything near Hudavendigar Cd. number 5' #there is no mention of romance istabul hotel even though we have specifically mentioned address 
## of that hotel
found_docs = qdrant.similarity_search(query,k=15)
for i in found_docs:
    print(i,'\n')
    break #remove the break to see complete results

page_content="hotel_name: Crowne Plaza Times Square Manhattan, an IHG Hotel\nhotel_description: Nestled in the heart of Midtown, our hotel puts you directly on Broadway between 48th and 49th Streets at the doorstep of Times Square. Stay steps from the greatest musical theater in the world, surrounded by iconic New York landmarks, top Fortune 500 companies, and the city's best shopping. Perfecting the art of blended travel, our hotel puts you blocks from Central Park, Rockefeller Center, and Radio City Music Hall. When you're not exploring or heading to a meeting, unwind in spacious rooms with all the essentials, including workspaces.  Relax, take in the city, and leave the details to us.\nreview_title: THEY DID NOT REFUND MY SECURITY DEPOSIT !\nreview_text: They did return my security deposit .They charged us 50 dollars per night !   Charged on 4th october till 9th October . I stayed 5 nights under the name Raafay Israr & Mahnoor Fayyaz . Its been over a month now . Need to resolve thi

### So the moment we tilted a query by a bit we see a dip in results although it's for one query 

## Phase-2

## connecting ares api

In [30]:
#not saving this key in .env because these jeys are not relaible
ares_api = "ares_adcdf99a238a2b1fc07252165453956ba2ed27e58c290903f6e5030f5c00058d"
ares_api

'ares_adcdf99a238a2b1fc07252165453956ba2ed27e58c290903f6e5030f5c00058d'

In [54]:

demo_ephemeral_chat_history.add_user_message('any image sof the hotel?')

result = document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)

In [55]:
result

"Romance Istanbul Hotel is a stylish and elegantly furnished hotel located at the intersection of Istanbul's most important old city area. It's known for its exceptional design, Turkish hospitality, and personal service. The hotel has 39 rooms, each equipped with a private bathroom, shower, satellite TV, and free wifi. The room size ranges from 20 m2 to 45 m2, and it includes various suite rooms and luxury room options.\n\nThe hotel has received excellent reviews, with guests praising the clean rooms, friendly and helpful staff, delicious breakfast, superb location, and the overall experience of staying at the hotel. The average rating value is 5.0, and there are 4023 reviews available.\n\nYou can find an image of the hotel on its TripAdvisor page: [Romance Istanbul Hotel](https://media-cdn.tripadvisor.com/media/photo-s/1c/02/78/ba/romance-istanbul-hotel.jpg)\n\nFeel free to reach out if you have more questions!"

In [42]:
# %%capture
# !pip install outlines #install this library for structured output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [47]:
dataset['train'][0]

{'hotel_name': 'Romance Istanbul Hotel',
 'hotel_description': 'Romance Istanbul Hotel has 39 rooms.Every room is elegantly furnished and harmonizes the modern life style with the traditional Ottoman touch. Romance Istanbul sits at the intersection of the old city’s most important part. With its luxuriously inspiring design and landmark old city location, steeped in the history of its surroundings, Romance Istanbul Hotel welcomes you with exceptional designed rooms and world-renowned Turkish hospitality. Our colleagues deliver the most personal service. It is perfectly placed and perfectly designed to enhance all that Istanbul has to offer. Each room offers a private bathroom and shower. Each is equipped with a satellite TV and free wifi connection. The rooms size change between 20 m2 and 45 m2. It includes 7 suite rooms: 1 Royal Suite, 4 Grand Suite, 1 Romance Suite and 1 Premium Suite, 2 Luxury Room With Terrace, 22 Deluxe Room, 8 City Room.',
 'review_title': 'An exceptional boutiqu

In [44]:
from enum import Enum
from pydantic import BaseModel, constr

import outlines
import torch

In [45]:
from outlines import models

model = models.openai("gpt-3.5-turbo",api_key=OPENAI_API_KEY)

In [48]:
class hotel_info(str, Enum):
    name_of_hotel = "Romance Istanbul Hotel"
    hotel_url = "https://www.tripadvisor.com/Hotel_Review-g293974-d8364987-Reviews-Romance_Istanbul_Hotel-Istanbul.html"
    hotel_image = "https://media-cdn.tripadvisor.com/media/photo-s/1c/02/78/ba/romance-istanbul-hotel.jpg"
    address = "Hudavendigar Cd. No:5 Sirkeci"
    country = "Turkiye"
    locality = "Istanbul"
    

In [49]:

# Construct structured sequence generator
generator = outlines.generate.json(model, hotel_info)


NotImplementedError: Cannot use JSON Schema-structure generation with an OpenAI model due to the limitations of the OpenAI API

In [ ]:

# Draw a sample
rng = torch.Generator(device="cuda")
rng.manual_seed(789001)

character = generator("Give me a character description", rng=rng)

print(repr(character))

In [35]:
import requests

url = "https://api-ares.traversaal.ai/live/predict"

payload = { "query": ["taco spots in San Francisco"] }
headers = {
  "x-api-key": ares_api,
  "content-type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

In [37]:
response.json()

'{"data":[{"response_text":"Here are some of the best taco spots in San Francisco:\\n\\n1. Al Pastor Papi: Known for their incredible fish tacos, try the baja taco with deep-fried rock cod and chipotle remoulade.\\n\\n2. Nopalito: A modern, all-organic Mexican restaurant offering make-it-yourself-style tacos with delicious house-made tortillas.\\n\\n3. Barrio: Enjoy stunning views of the Golden Gate Bridge while indulging in their Cochinita Pibil tacos.\\n\\n4. Bonita Taqueria and Rotisserie: Hidden gem serving up tasty mahi-mahi tacos with tangy coleslaw and arbol chile oil.\\n\\n5. Cielito Lindo: Visit this authentic taqueria for their mouthwatering quesa birria tacos and rich consomme.\\n\\n6. Lolo\'s: Located in the Mission District, Lolo\'s offers a range of exciting tacos that will satisfy any palate.\\n\\nPlease note that these are just a few recommendations and there are many more great taco spots to explore in San Francisco.","web_url":["https://sf.eater.com/maps/best-tacos-sa